In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, Concatenate, Input
from keras.callbacks import EarlyStopping
import word2vec_utils as w2v
import data
import numpy as np
import re
from data_utils import split_dataset 
from chat_constants import *


Using Theano backend.


In [10]:
model = w2v.initialize()

In [8]:
A1, B, A2 = data.load_data()


In [4]:
A1_train,B_train,A2_train = w2v.get_n_training_data(20000,A1,B,A2)
A1B_train = np.concatenate((A1_train,B_train),axis=1)

In [4]:
def null_punisher(y_true, y_pred ):
    '''Provides stronger incentive to avoid Null'''
    L = 3
    error = keras.losses.cosine_proximity( y_true,y_pred)
    if y_true != BLANK:
        error *= L
    return error

In [5]:
LSTM_DROPOUT = 0.1

In [ ]:
A1_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

A1_layer1 = LSTM(EMBED_DIM,return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A1 = A1_layer1(A1_input)

In [ ]:
B_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

B_layer1 = LSTM(EMBED_DIM,return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
B = B_layer1(B_input)

In [ ]:
combined = Concatenate([A1,B])


In [6]:
AB_Input = Input(shape =(MAX_SENT_LENGTH*2,EMBED_DIM))

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer3 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer3", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer3(AB_layer2(AB_layer1(AB_Input)))

chat_model = Model(inputs = [ AB_Input], outputs = [AB_output])
chat_model.compile(loss=null_punisher,
              optimizer='Adam',
              metrics=['accuracy']
              )
early_stopping = EarlyStopping( monitor = 'val_acc',patience = 2)

In [8]:
chat_model.load_weights('chat_net.h5',by_name=True)
chat_model.fit([A1B_train], A2_train,
          batch_size=200, epochs=20,
          validation_split = 0.025 )
chat_model.save('chat_net.h5')

Train on 19500 samples, validate on 500 samples
Epoch 1/20
19500/19500 [==============================] - 785s - loss: -0.0098 - acc: 0.8399 - val_loss: -0.0099 - val_acc: 0.8833
Epoch 2/20
19500/19500 [==============================] - 1081s - loss: -0.0099 - acc: 0.8843 - val_loss: -0.0099 - val_acc: 0.9500
Epoch 3/20
19500/19500 [==============================] - 1644s - loss: -0.0099 - acc: 0.9070 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 4/20
19500/19500 [==============================] - 1811s - loss: -0.0099 - acc: 0.9171 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 5/20
19500/19500 [==============================] - 2031s - loss: -0.0099 - acc: 0.9234 - val_loss: -0.0099 - val_acc: 0.9833
Epoch 6/20
19500/19500 [==============================] - 1982s - loss: -0.0099 - acc: 0.9287 - val_loss: -0.0099 - val_acc: 0.9667
Epoch 7/20
19500/19500 [==============================] - 1972s - loss: -0.0099 - acc: 0.9333 - val_loss: -0.0099 - val_acc: 1.0000
Epoch 8/20
19500/19500 [=====

In [25]:
chat_model.load_weights('chat_net.h5',by_name=True)

A1_test,B_test,A2_test = w2v.get_n_training_data(2,A1[10000:],B[10000:],A2[10000:])
A1B_test = np.concatenate((A1_test,B_test),axis=1)
print A1B_test

[[[ 0.05078125 -0.09326172  0.06494141 ..., -0.04150391  0.00817871  0.        ]
  [ 0.140625   -0.02722168  0.03295898 ..., -0.08935547  0.10205078  0.        ]
  [ 0.05078125 -0.09326172  0.06494141 ..., -0.04150391  0.00817871  0.        ]
  ..., 
  [ 0.          0.          0.         ...,  0.          0.          1.        ]
  [ 0.          0.          0.         ...,  0.          0.          1.        ]
  [ 0.          0.          0.         ...,  0.          0.          1.        ]]

 [[ 0.05078125 -0.09326172  0.06494141 ..., -0.04150391  0.00817871  0.        ]
  [ 0.140625   -0.02722168  0.03295898 ..., -0.08935547  0.10205078  0.        ]
  [ 0.05078125 -0.09326172  0.06494141 ..., -0.04150391  0.00817871  0.        ]
  ..., 
  [ 0.          0.          0.         ...,  0.          0.          1.        ]
  [ 0.          0.          0.         ...,  0.          0.          1.        ]
  [ 0.          0.          0.         ...,  0.          0.          1.        ]]]


In [13]:
w2v.unvectorize_initialize()

In [26]:
predicted = chat_model.predict(A1B_test)

In [27]:
predicted_words = w2v.unvectorize_sentence( predicted[0] )
print( predicted_words )

('w2v word-score', u'1', 0.9980672597885132, '\n', 'uk word-score', 'a', 0.056033043745772247)
('w2v word-score', u'@', 0.9984095692634583, '\n', 'uk word-score', 'break-', 0.052939259401523289)
('w2v word-score', u'Not', 0.9968540668487549, '\n', 'uk word-score', 'and', 0.082096403343257435)
('w2v word-score', u'the', 0.9954041242599487, '\n', 'uk word-score', 'a', 0.072104772078892515)
('w2v word-score', u'hacking', 0.9770256876945496, '\n', 'uk word-score', 'to', 0.076789168099464206)
('w2v word-score', u'Darmer', 0.315388947725296, '\n', 'uk word-score', 'and', 0.97905361822752179)
('w2v word-score', u'gagging', 0.8061763048171997, '\n', 'uk word-score', 'and', 0.5102656647626258)
('w2v word-score', u'Regal_Beagle', 0.31222817301750183, '\n', 'uk word-score', 'and', 0.96898525661626256)
('w2v word-score', u'spitting', 0.9452880620956421, '\n', 'uk word-score', 'and', 0.23605261754542997)
('w2v word-score', u'part', 0.9399054050445557, '\n', 'uk word-score', '.', 0.16153080103069159

In [28]:
for word in predicted[0]:
    print word[-1]

4.92014e-06
4.56457e-05
0.000285789
0.000822868
0.00136276
0.00204014
0.00225027
-0.000994075
-0.00131872
-0.00209939
-0.0021074
-0.00377722
-0.00487315
-0.000230759
0.108067
0.724087
0.939856
0.9877
0.996966
0.968217
0.865467
0.803786
0.839297
0.99998
0.999996
0.999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [12]:
a = {}
a['__BLANK__'] = BLANK
a['__BLANK__']

import pickle
with open('unknown_words_stored.pkl', 'w+') as f:
    pickle.dump(a, f)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1]

In [17]:
print( A1[30000], B[30000],A2[30000] )

('1 9 1 @ Yes I am .', '1 9 1 @ What are you doing ?', '1 9 1 @ Nothing .')


In [11]:
len(A1_train[1])

30

In [41]:
len(A1)

106455

In [4]:
A2.shape

NameError: name 'A2' is not defined